# Projeto Final - Aprendizado de Máquina
## Análise do Dataset SECOM

Este notebook implementa o pipeline solicitado no trabalho final da disciplina.

### Estrutura:
1. Carregamento e exploração dos dados (EDA)
2. Pré-processamento (imputação, escalonamento, remoção de colunas constantes)
3. Redução de dimensionalidade (PCA/LDA)
4. Modelagem
5. Resultados e Comparação
6. Resultados e Comparação
7. Ajustando limiar de decisão do melhor modelo
7. Conclusões


In [ ]:
# Importação de bibliotecas principais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, roc_auc_score, balanced_accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.pipeline import Pipeline as ImbPipeline # Pipeline especial para SMOTE
from imblearn.over_sampling import SMOTE
from sklearn.metrics import average_precision_score,precision_recall_curve


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from lightgbm import LGBMClassifier


## 1. Carregamento e Exploração dos Dados (EDA)
Carregaremos o dataset SECOM.
- Arquivo `secom.data` contém as features
- Arquivo `secom_labels.data` contém as labels (-1 = Pass, 1 = Fail)

Além disso, já e feito a divisão entre treino e teste para evitar data leakage.

In [ ]:
# Ajuste os caminhos para os arquivos antes de executar
X = pd.read_csv('../data/raw/secom.data', sep=' ', header=None, na_values='NaN')
# Converta os nomes para string para evitar ambiguidades
X.columns = X.columns.astype(str)
y = pd.read_csv('../data/raw/secom_labels.data', sep=' ', header=None, usecols=[0])
y = y[0]

print("Shape X:", X.shape)
print("Shape y:", y.shape)
print("Classes:", y.value_counts())

X.head()

In [ ]:
# Explorando quantidade de NA's em cada coluna
X.isna().mean().sort_values(ascending=False).head(40)


In [ ]:
X.dtypes.describe()

In [ ]:
# O split é feito ANTES de qualquer processamento para simular dados "novos" no teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

## 2. Pré-processamento
- Removendo variáveis com mais de 40% de valores faltantes
- Imputação de valores faltantes (média)
- Remoção de colunas constantes
- Padronização (z-score)


In [ ]:
# Classe para remoção de colunas com 40 de NA

class ColumnDropperByNa(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.4):
        self.threshold = threshold
        self.columns_to_keep_ = []

    def fit(self, X, y=None):
        # Aprende quais colunas manter APENAS dos dados de treino (X)
        nan_frac = X.isna().mean()
        self.columns_to_keep_ = X.columns[nan_frac < self.threshold].tolist()
        return self

    def transform(self, X, y=None):
        # Aplica a transformação, mantendo apenas as colunas aprendidas
        return X[self.columns_to_keep_]

In [ ]:
# -----------------------
# Criar pipeline de pré processamento inical
# -----------------------

preprocessor_pipeline = Pipeline(steps=[
    ('remover_por_na', ColumnDropperByNa(threshold=0.4)),
    ('imputer', SimpleImputer(strategy="median")),
    ('remover_constantes', VarianceThreshold()),
    ('scaler', StandardScaler())
])

# -----------------------
# Aplicar transformação aos dados
# -----------------------
X_train_transformed = preprocessor_pipeline.fit_transform(X_train)

# Voltar para DataFrame
X_train_prepared = pd.DataFrame(X_train_transformed)

print("Shape antes:", X_train.shape)
print("Shape depois:", X_train_prepared.shape)
print(X_train_prepared.head())


## 3. Redução de Dimensionalidade (PCA/LDA)
Aplicaremos PCA para reduzir a dimensionalidade e visualizar os dados.


In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_train_prepared)

plt.figure(figsize=(8,6))
sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=y_train, palette='coolwarm', alpha=0.7)
plt.title('Projeção PCA (2 componentes)')
plt.show()


A análise via PCA demonstrou que as direções de maior variância nos dados não são as mesmas que separam eficientemente as classes de operação normal das falhas, resultando em uma sobreposição visual significativa. Isso ocorre porque o PCA é uma técnica não supervisionada, que ignora os rótulos ao projetar os dados. Diante disso, a utilização da Análise de Discriminante Linear (LDA) é a próxima etapa lógica. Diferentemente do PCA, o LDA é um método supervisionado cujo objetivo é encontrar a projeção dos dados que maximiza a separação entre as classes. Portanto, ao invés de buscar a variância, o LDA buscará o eixo que melhor distingue os grupos, o que pode revelar uma estrutura de separação que não era visível com o PCA.

In [ ]:
# Como há 2 classes, usamos n_components=1
lda = LinearDiscriminantAnalysis(n_components=1)
X_lda = lda.fit_transform(X_train_prepared, y_train) # Note que 'y' é passado aqui!

# Para visualizar, podemos plotar a densidade de cada classe
df_lda = pd.DataFrame({'LD1': X_lda.flatten(), 'label': y_train})
sns.displot(df_lda, x='LD1', hue='label', kind='kde', fill=True)
plt.title('Separação das Classes usando LDA')
plt.show()

A análise com LDA (Análise de Discriminante Linear) foi bem-sucedida, demonstrando, ao contrário do PCA, que o problema é amplamente separável. A técnica projetou os dados de alta dimensão em um único eixo (LD1) no qual as duas classes apresentam distribuições claramente distintas: a classe normal (-1) forma um grupo denso e consistente, enquanto a classe de falha (1) se concentra em uma região separada, embora com maior variabilidade. Este resultado é extremamente promissor, pois confirma que um modelo de classificação pode aprender uma fronteira linear para distinguir os casos com alta eficácia, transformando um problema complexo em um que é visivelmente mais simples de resolver.

## 4. Modelagem
Usaremos Logistic Regression, Random Forest e SVM.
- Cada um com pelo menos 2 variações de hiperparâmetros via GridSearch.
- Validação com Stratified 10-Fold Cross Validation.


### Modelagem sem LDA

In [ ]:
# ==============================================================================
# 1. DEFINIÇÃO DOS CENÁRIOS DE PRÉ-PROCESSAMENTO
# ==============================================================================

# Definimos os dois pré-processadores base
preprocessor_base = Pipeline(steps=[
    ('remover_por_na', ColumnDropperByNa(threshold=0.4)),
    ('imputer', SimpleImputer(strategy="median")),
    ('remover_constantes', VarianceThreshold()),
    ('scaler', StandardScaler())
])

preprocessor_lda = Pipeline(steps=[
    ('remover_por_na', ColumnDropperByNa(threshold=0.4)),
    ('imputer', SimpleImputer(strategy="median")),
    ('remover_constantes', VarianceThreshold()),
    ('scaler', StandardScaler()),
    ('lda', LinearDiscriminantAnalysis(n_components=1))
])

# Agora criamos uma lista de configurações para cada cenário que queremos testar
scenarios = [
    {'name': 'Sem LDA, Sem SMOTE', 'preprocessor': preprocessor_base, 'use_smote': False},
    {'name': 'Com LDA, Sem SMOTE', 'preprocessor': preprocessor_lda, 'use_smote': False},
    {'name': 'Sem LDA, Com SMOTE', 'preprocessor': preprocessor_base, 'use_smote': True},
    {'name': 'Com LDA, Com SMOTE', 'preprocessor': preprocessor_lda, 'use_smote': True},
]

# ==============================================================================
# 2. DEFINIÇÃO ÚNICA DOS MODELOS E PARÂMETROS
# ==============================================================================
# (Mesma definição de antes)
models_and_params = {
    'LogisticRegression': {
        'model': LogisticRegression(max_iter=2000, random_state=42, class_weight='balanced'),
        'params': { 'model__C': [0.01, 0.1, 1, 10], 'model__penalty': ['l2'] }
    },
    'RandomForest': {
        'model': RandomForestClassifier(random_state=42, class_weight='balanced'),
        'params': { 'model__n_estimators': [100, 200], 'model__max_depth': [10, 20], 'model__min_samples_leaf': [1, 5] }
    },
    'SVM': {
        'model': SVC(probability=True, random_state=42, class_weight='balanced'),
        'params': { 'model__C': [0.1, 1, 10], 'model__kernel': ['linear', 'rbf'], 'model__gamma': ['scale', 'auto'] }
    },
    'LightGBM': {
        'model': LGBMClassifier(random_state=42),
        'params': { 'model__n_estimators': [100, 200], 'model__learning_rate': [0.05, 0.1], 'model__scale_pos_weight': [10, 20, 30] }
    }
}

# ==============================================================================
# 3. LOOP ÚNICO DE TREINAMENTO E AVALIAÇÃO
# ==============================================================================
all_results = []
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# ==============================================================================
# 3. LOOP ÚNICO DE TREINAMENTO E AVALIAÇÃO (CORRIGIDO)
# ==============================================================================
all_results = []
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for scenario in scenarios:
    scenario_name = scenario['name']
    preprocessor = scenario['preprocessor']
    use_smote = scenario['use_smote']
    
    print(f"--- EXECUTANDO CENÁRIO: {scenario_name} ---")
    
    for model_name, config in models_and_params.items():
        print(f"\nTreinando {model_name}...")
        
        # --- INÍCIO DA CORREÇÃO ---
        # Em vez de começar com o pipeline, começamos com a lista de passos DENTRO dele
        pipeline_steps = preprocessor.steps.copy() 
        # --- FIM DA CORREÇÃO ---
        
        model = config['model']
        params = config['params'].copy()
        
        if use_smote:
            pipeline_steps.append(('smote', SMOTE(random_state=42)))
            if 'class_weight' in model.get_params():
                model.set_params(class_weight=None)
            if 'model__scale_pos_weight' in params:
                del params['model__scale_pos_weight']
        
        pipeline_steps.append(('model', model))
        
        if use_smote:
            full_pipeline = ImbPipeline(steps=pipeline_steps)
        else:
            full_pipeline = Pipeline(steps=pipeline_steps)
            
        grid = GridSearchCV(full_pipeline, params, cv=cv, scoring='roc_auc', n_jobs=-1, verbose=0)
        grid.fit(X_train, y_train)
        
        best_model = grid.best_estimator_
        y_pred = best_model.predict(X_test)
        y_prob = best_model.predict_proba(X_test)[:, 1]

        all_results.append({
            'scenario': scenario_name,
            'model': model_name,
            'roc_auc_test': roc_auc_score(y_test, y_prob),
            'auprc_test': average_precision_score(y_test, y_prob), 
            'balanced_accuracy_test': balanced_accuracy_score(y_test, y_pred),
            'f1_test': f1_score(y_test, y_pred, pos_label=1),
            'confusion_matrix': confusion_matrix(y_test, y_pred),
            'best_params': grid.best_params_
        })

## 5. Resultados e Comparação
Aqui apresentamos os resultados de cada classificador no conjunto de teste.


In [ ]:
# Converter a lista de resultados em um DataFrame do Pandas
results_df = pd.DataFrame(all_results)

# 1. Tabela Comparativa usando AUPRC (ou Balanced Accuracy)
print("\n## Tabela Comparativa (AUPRC no Teste) ##")
comparison_table = results_df.pivot_table(
    index='model', 
    columns='scenario', 
    values='auprc_test' # <-- MUDANÇA PRINCIPAL AQUI
)

# Reordena as colunas para uma melhor visualização lógica
comparison_table = comparison_table[['Sem LDA, Sem SMOTE', 'Sem LDA, Com SMOTE', 'Com LDA, Sem SMOTE', 'Com LDA, Com SMOTE']]
print(comparison_table.round(4))

# 2. Detalhes e Matriz de Confusão para cada experimento
for result in all_results:
    print("\n" + "="*80)
    print(f"## Modelo: {result['model']} | Cenário: {result['scenario']} ##")
    print(f"ROC AUC (Teste): {result['roc_auc_test']:.4f}")
    print(f"Melhores Hiperparâmetros: {result['best_params']}")
    
    plt.figure(figsize=(6, 4))
    sns.heatmap(result['confusion_matrix'], annot=True, fmt='d', cmap='Blues')
    plt.title(f"Matriz de Confusão - {result['model']} ({result['scenario']})")
    plt.ylabel('Label Verdadeiro')
    plt.xlabel('Label Previsto')
    plt.show()

## 6. Ajustando limiar de decisão do melhor modelo


In [ ]:
# ==============================================================================
# PASSO 1: SELECIONAR E RETREINAR O MODELO CAMPEÃO 
# (Esta parte já funcionou para você, mantida para contexto)
# ==============================================================================
print("--- Iniciando o ajuste do limiar para o melhor modelo ---")

CENARIO_VENCEDOR = 'Sem LDA, Sem SMOTE'
MODELO_VENCEDOR = 'RandomForest'

try:
    best_model_config = next(item for item in all_results if item["model"] == MODELO_VENCEDOR and item["scenario"] == CENARIO_VENCEDOR)
except (NameError, StopIteration):
    print("ERRO: A variável 'all_results' não foi encontrada ou o modelo/cenário vencedor não está nela.")
    exit()

best_params = best_model_config['best_params']
model_params = {key.replace('model__', ''): value for key, value in best_params.items()}

best_pipeline = Pipeline([
    ('preprocessor', preprocessor_pipeline),
    ('model', RandomForestClassifier(random_state=42, class_weight='balanced', **model_params))
])

print("Retreinando o modelo campeão com os melhores parâmetros...")
best_pipeline.fit(X_train, y_train)

# ==============================================================================
# PASSO 2, 3: OBTER PROBABILIDADES E PADRONIZAR RÓTULOS
# (Mantidos como antes)
# ==============================================================================
y_probs_train = best_pipeline.predict_proba(X_train)[:, 1]
y_probs_test = best_pipeline.predict_proba(X_test)[:, 1]
y_train_mapped = y_train.replace({-1: 0})
y_test_mapped = y_test.replace({-1: 0})
print("Probabilidades previstas e rótulos mapeados.")

# ==============================================================================
# PASSO 4: ENCONTRAR O LIMIAR ÓTIMO (Lógica Levemente Ajustada)
# ==============================================================================
precision, recall, thresholds = precision_recall_curve(y_train_mapped, y_probs_train)

# Calcula o F1-score para cada limiar
f1_scores = 2 * (precision * recall) / (precision + recall)
f1_scores = np.nan_to_num(f1_scores)

# Encontra o melhor limiar (considerando apenas os N primeiros scores)
best_f1_idx = np.argmax(f1_scores[:-1])
best_threshold = thresholds[best_f1_idx]

print(f"\nMelhor F1-Score (no treino): {f1_scores[best_f1_idx]:.4f}")
print(f"Limiar ótimo encontrado: {best_threshold:.4f}")

# ==============================================================================
# PASSO 5: VISUALIZAR O TRADE-OFF (CORRIGIDO)
# ==============================================================================
plt.figure(figsize=(10, 6))
# Usamos thresholds como eixo X e fatiamos os outros arrays para terem o mesmo tamanho
plt.plot(thresholds, precision[:-1], label='Precision', color='blue')
plt.plot(thresholds, recall[:-1], label='Recall', color='green')
plt.plot(thresholds, f1_scores[:-1], label='F1-Score', color='red', linestyle='--') # <-- CORREÇÃO APLICADA
plt.axvline(x=best_threshold, color='black', linestyle=':', label=f'Limiar Ótimo ({best_threshold:.2f})')
plt.title('Precision, Recall e F1-Score vs. Limiar de Decisão (Dados de Treino)')
plt.xlabel('Limiar de Probabilidade')
plt.ylabel('Score')
plt.legend()
plt.grid(True)
plt.show()

# ==============================================================================
# PASSO 6: AVALIAR O DESEMPENHO FINAL NO CONJUNTO DE TESTE
# (Mantido como antes)
# ==============================================================================
y_pred_test_optimized = (y_probs_test >= best_threshold).astype(int)

cm_optimized = confusion_matrix(y_test_mapped, y_pred_test_optimized)
new_f1_score = f1_score(y_test_mapped, y_pred_test_optimized)
new_ba_score = balanced_accuracy_score(y_test_mapped, y_pred_test_optimized)

print("\n--- RESULTADOS FINAIS NO CONJUNTO DE TESTE (COM LIMIAR OTIMIZADO) ---")
print(f"F1-Score: {new_f1_score:.4f}")
print(f"Balanced Accuracy: {new_ba_score:.4f}")

plt.figure(figsize=(6, 4))
sns.heatmap(cm_optimized, annot=True, fmt='d', cmap='Greens',
            xticklabels=['Previsto Normal (0)', 'Previsto Falha (1)'],
            yticklabels=['Real Normal (0)', 'Real Falha (1)'])
plt.title(f'Matriz de Confusão Final (Limiar = {best_threshold:.2f})')
plt.ylabel('Label Verdadeiro')
plt.xlabel('Label Previsto')
plt.show()

In [ ]:
# Use apenas o pipeline do RandomForest Sem LDA, Sem SMOTE
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor_pipeline),
    ('model', RandomForestClassifier(random_state=42, class_weight='balanced'))
])

# Nova grade de parâmetros focada em regularização
params_rf_regularized = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [3, 5, 7, 10], # Teste árvores BEM mais simples
    'model__min_samples_leaf': [10, 20, 50], # Force as folhas a serem maiores
    'model__max_features': ['sqrt', 0.3, 0.5] # Limite as features por árvore
}

print("\n--- INICIANDO NOVO TREINO DO RANDOMFOREST COM REGULARIZAÇÃO FORTE ---")
grid_rf_reg = GridSearchCV(rf_pipeline, params_rf_regularized, cv=cv, scoring='roc_auc', n_jobs=-1, verbose=1)
grid_rf_reg.fit(X_train, y_train)

# Após o treino, você deve repetir o processo de ajuste de limiar com este novo `grid_rf_reg.best_estimator_`
# e avaliar os resultados no teste.

In [ ]:
print("\n--- Iniciando o ajuste do limiar para o NOVO modelo regularizado ---")

# ==============================================================================
# PASSO 1: OBTER O MELHOR ESTIMADOR DO NOVO GRIDSEARCH
# ==============================================================================
# Pegamos o melhor pipeline encontrado pelo novo GridSearchCV
best_regularized_model = grid_rf_reg.best_estimator_
print("Melhor modelo regularizado selecionado.")
print("Melhores parâmetros encontrados:", grid_rf_reg.best_params_)

# ==============================================================================
# PASSO 2: OBTER PROBABILIDADES
# ==============================================================================
# Prevemos as probabilidades para os conjuntos de TREINO e TESTE
y_probs_train_reg = best_regularized_model.predict_proba(X_train)[:, 1]
y_probs_test_reg = best_regularized_model.predict_proba(X_test)[:, 1]
print("Probabilidades previstas para treino e teste.")

# ==============================================================================
# PASSO 3: PADRONIZAR OS RÓTULOS PARA {0, 1} (se ainda não tiver feito)
# ==============================================================================
y_train_mapped = y_train.replace({-1: 0})
y_test_mapped = y_test.replace({-1: 0})
print("Rótulos de treino e teste mapeados para {0, 1}.")

# ==============================================================================
# PASSO 4: ENCONTRAR O LIMIAR ÓTIMO NO CONJUNTO DE TREINO
# ==============================================================================
precision_reg, recall_reg, thresholds_reg = precision_recall_curve(y_train_mapped, y_probs_train_reg)

# Calcula o F1-score para cada limiar
f1_scores_reg = 2 * (precision_reg * recall_reg) / (precision_reg + recall_reg)
f1_scores_reg = np.nan_to_num(f1_scores_reg)

# Encontra o melhor limiar
best_f1_idx_reg = np.argmax(f1_scores_reg[:-1])
best_threshold_reg = thresholds_reg[best_f1_idx_reg]

print(f"\nMelhor F1-Score (no treino): {f1_scores_reg[best_f1_idx_reg]:.4f}")
print(f"Limiar ótimo encontrado: {best_threshold_reg:.4f}")

# ==============================================================================
# PASSO 5: VISUALIZAR O TRADE-OFF
# ==============================================================================
plt.figure(figsize=(10, 6))
plt.plot(thresholds_reg, precision_reg[:-1], label='Precision')
plt.plot(thresholds_reg, recall_reg[:-1], label='Recall')
plt.plot(thresholds_reg, f1_scores_reg[:-1], label='F1-Score', linestyle='--')
plt.axvline(x=best_threshold_reg, color='black', linestyle=':', label=f'Limiar Ótimo ({best_threshold_reg:.2f})')
plt.title('Trade-off do Modelo Regularizado (Dados de Treino)')
plt.xlabel('Limiar de Probabilidade')
plt.ylabel('Score')
plt.legend()
plt.grid(True)
plt.show()

# ==============================================================================
# PASSO 6: AVALIAR O DESEMPENHO FINAL NO CONJUNTO DE TESTE
# ==============================================================================
# Aplica o melhor limiar nas probabilidades do conjunto de teste
y_pred_test_optimized_reg = (y_probs_test_reg >= best_threshold_reg).astype(int)

# Gera a nova matriz de confusão e métricas
cm_optimized_reg = confusion_matrix(y_test_mapped, y_pred_test_optimized_reg)
new_f1_score_reg = f1_score(y_test_mapped, y_pred_test_optimized_reg)
new_ba_score_reg = balanced_accuracy_score(y_test_mapped, y_pred_test_optimized_reg)

print("\n--- RESULTADOS FINAIS NO TESTE (MODELO REGULARIZADO E LIMIAR OTIMIZADO) ---")
print(f"F1-Score: {new_f1_score_reg:.4f}")
print(f"Balanced Accuracy: {new_ba_score_reg:.4f}")

# Plotar a Matriz de Confusão Final
plt.figure(figsize=(6, 4))
sns.heatmap(cm_optimized_reg, annot=True, fmt='d', cmap='Oranges',
            xticklabels=['Previsto Normal (0)', 'Previsto Falha (1)'],
            yticklabels=['Real Normal (0)', 'Real Falha (1)'])
plt.title(f'Matriz de Confusão Final (Limiar = {best_threshold_reg:.2f})')
plt.ylabel('Label Verdadeiro')
plt.xlabel('Label Previsto')
plt.show()

## 7. Tentando um modelo mais simples (Regressão logistica)

In [ ]:
# ==============================================================================
# PASSO 1: SETUP E DEFINIÇÃO DOS EXPERIMENTOS
# ==============================================================================

scenarios_to_tune = ['Sem LDA, Com SMOTE', 'Com LDA, Com SMOTE']
MODELO_ALVO = 'LogisticRegression'

y_train_mapped = y_train.replace({-1: 0})
y_test_mapped = y_test.replace({-1: 0})

final_tuned_results = []

print(f"--- Iniciando ajuste de limiar para {MODELO_ALVO} nos cenários com SMOTE ---")

# ==============================================================================
# PASSO 2: LOOP PARA AJUSTAR O LIMIAR E AVALIAR CADA CENÁRIO (CORRIGIDO)
# ==============================================================================

for scenario_name in scenarios_to_tune:
    print("\n" + "="*80)
    print(f"PROCESSANDO CENÁRIO: {scenario_name}")
    
    try:
        model_config = next(item for item in all_results if item["model"] == MODELO_ALVO and item["scenario"] == scenario_name)
    except (NameError, StopIteration):
        print(f"ERRO: Configuração para '{scenario_name}' não encontrada. Pulando.")
        continue
        
    best_params = model_config['best_params']
    model_params = {key.replace('model__', ''): value for key, value in best_params.items()}
    
    preprocessor = preprocessor_lda if 'Com LDA' in scenario_name else preprocessor_base
    
    # --- INÍCIO DA CORREÇÃO ---
    # 1. Copiamos os passos do pré-processador para uma nova lista
    pipeline_steps = preprocessor.steps.copy()
    
    # 2. Adicionamos os passos de SMOTE e do modelo a essa lista "plana"
    pipeline_steps.append(('smote', SMOTE(random_state=42)))
    pipeline_steps.append(('model', LogisticRegression(max_iter=2000, random_state=42, **model_params)))
    
    # 3. Criamos o ImbPipeline a partir da lista de passos desempacotada
    final_pipeline = ImbPipeline(pipeline_steps)
    # --- FIM DA CORREÇÃO ---
    
    print("Treinando o pipeline final...")
    final_pipeline.fit(X_train, y_train)
    
    # --- Encontra o limiar ótimo nos dados de TREINO ---
    y_probs_train = final_pipeline.predict_proba(X_train)[:, 1]
    precision, recall, thresholds = precision_recall_curve(y_train_mapped, y_probs_train)
    
    f1_scores = 2 * (precision * recall) / (precision + recall)
    f1_scores = np.nan_to_num(f1_scores)
    
    best_f1_idx = np.argmax(f1_scores[:-1])
    best_threshold = thresholds[best_f1_idx]
    print(f"Limiar ótimo encontrado (F1-Score treino): {best_threshold:.4f}")
    
    # --- Avalia o desempenho no TESTE com o limiar ótimo ---
    y_probs_test = final_pipeline.predict_proba(X_test)[:, 1]
    y_pred_optimized = (y_probs_test >= best_threshold).astype(int)
    
    final_f1 = f1_score(y_test_mapped, y_pred_optimized)
    final_ba = balanced_accuracy_score(y_test_mapped, y_pred_optimized)
    final_cm = confusion_matrix(y_test_mapped, y_pred_optimized)
    
    final_tuned_results.append({
        'scenario': scenario_name,
        'model': MODELO_ALVO,
        'final_f1_score': final_f1,
        'final_balanced_accuracy': final_ba,
        'best_threshold': best_threshold,
        'confusion_matrix': final_cm
    })
    
    plt.figure(figsize=(6, 4))
    sns.heatmap(final_cm, annot=True, fmt='d', cmap='viridis')
    plt.title(f'Matriz de Confusão Final - {scenario_name}')
    plt.ylabel('Label Verdadeiro')
    plt.xlabel('Label Previsto')
    plt.show()

# ==============================================================================
# PASSO 3: APRESENTAR A COMPARAÇÃO FINAL
# ==============================================================================

print("\n" + "="*80)
print("## Tabela Comparativa Final (Após Ajuste de Limiar) ##")

summary_df_tuned = pd.DataFrame(final_tuned_results)
summary_df_tuned = summary_df_tuned.set_index('scenario')
print(summary_df_tuned[['final_f1_score', 'final_balanced_accuracy', 'best_threshold']].round(4))

## 8. Conclusões



A análise aprofundada dos resultados elege como campeão o modelo de **Regressão Logística** no cenário que combina o pré-processamento com **LDA e SMOTE**. Após um ajuste fino do limiar de decisão para otimizar o F1-Score, este modelo demonstrou ser a abordagem mais eficaz para superar o severo desbalanceamento dos dados e o overfitting, passando de uma performance nula na detecção de falhas para um **F1-Score final de 0.25** no conjunto de teste. Este resultado, embora modesto, representa um avanço crucial, transformando um problema inicialmente intratável em um modelo funcional com capacidade real de identificar defeitos que antes seriam completamente ignorados.